In [2]:
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [3]:
# check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# CONDITIONS
# the majority pretrained models require the input to be 224x224 images
# math the normalization
# color channel,the means are [0.485, 0.456, 0.406]
# standard derivation are [0.229, 0.224, 0.225]
#image = (image - mean) / std

data_dir = 'data/Cat_dog_data'
# Define transforms
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir + '/train', transform = train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test1', transform = test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size = 64, shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size = 64)

In [9]:
# get our pretrained network
model = models.resnet50(pretrained = True)
# change the last layer "fc"
# make sure our model parameters are frozen, so when training they don't get updated

# Turn off gradients for the model
for param in model.parameters():
    param.requires_grad = False
    
# DEFINE NEW CLASSIFIER
classifier = nn.Sequential(nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p = 0.2),
                          nn.Linear(512, 2),
                          nn.LogSoftmax(dim = 1))

# check the last layer (classifier) again
model.fc = classifier
# get loss
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr = 0.0003)

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [ ]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
    for images, labels in trainloader:
        steps += 1
        # move to GPU
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            # transfer to GPU
            images, labels = images.to(device), labels.to(device)
            
            # set evaluation model -> turn off dropout -> focus to make predictions instead test loss and accuracy
            model.eval()
            test_loss = 0
            accuracy = 0
            
            for images, labels in testloader:
                logps = model (images)
                loss = criterion(logps, labels)
                test_loss += loss.item()
                
                #calculate accuracy
                ps = torch.exp(logps)
                top_ps, top_class = ps.topk(1, dim = 1)
                equality = top_class = labels.view(*top_class.shape)
                accuracy += torch.mean(equality.type(torch.FloatTensor)).item()
                
            print(f"Epoch {epoch + 1}/{epochs}..."
                  f"Train loss: {running_loss/print_every:-3f}.."
                  f"Test loss: {test_loss/len(testloader): .3f}"
                  f"Test accuracy: {accuracy/len(testloader): .3f}")
                